# Взаимодействие с треками

Для взаимодействия с треками вам **необходим** корректный токен.

Получаем токен из окружения среды

In [1]:
val yamToken = System.getenv("YAM_TOKEN")!!

Инициализируем клиента

In [2]:
import su.pank.yamapi.builder.createYaMusicApiClient

val client = createYaMusicApiClient {
    token = yamToken
}

## Получение трека по его ID

Получение трека https://music.yandex.ru/album/22049037/track/728014

In [3]:
import kotlinx.coroutines.runBlocking

val track = runBlocking {
     client.tracks("728014").first()
}
track

Track(id=728014, title=La Maritza, available=true, availableForPremiumUsers=true, availableFullWithoutPermission=false, availableForOptions=[Bookmate], durationMs=220180, previewDurationMs=30000, storageDir=51327_109b74ca.36526310.1.609676, fileSize=0, r128=R128(i=-9.74, tp=0.7), artists=[Artist(id=18354, name=Sylvie Vartan, cover=Cover(type=FromAlbumCover, uri=avatars.yandex.net/get-music-content/6300975/8c474d84.a.22048939-1/%%, prefix=8c474d84.a.22048939-1, dir=null, version=null, custom=null), various=false, composer=false, genres=[])], albums=[Album(id=22048939, title='Triple Best Of', type=null, metaType=Music, year=2009, releaseDate=2009-11-16T21:00:00Z, coverUri=avatars.yandex.net/get-music-content/6300975/8c474d84.a.22048939-1/%%, ogImage='avatars.yandex.net/get-music-content/6300975/8c474d84.a.22048939-1/%%', genre=pop, trackCount=60, likesCount=154, recent=false, veryImportant=false, artists=[Artist(id=18354, name=Sylvie Vartan, cover=Cover(type=FromAlbumCover, uri=avatars.y

Как вы могли заметить, метод `tracks` принимает на вход **список** ID треков и возвращает **список** треков.

In [4]:
val tracks = runBlocking {
    client.tracks("62690680", "728014")
}
tracks.forEach {
    println(it.title + " - " + it.artists.joinToString(", ") { artist -> artist.name })
}

Voyage Voyage - Meimuna
La Maritza - Sylvie Vartan


## Скачивание трека

> Внимание! Скачивание треков может нарушать авторские права. Убедитесь, что у вас есть право на скачивание и использование треков в соответствии с законодательством вашей страны и условиями использования сервиса Яндекс Музыка.

Для скачивания трека нам понадобится его `downloadInfo`, который мы можем получить из объекта трека.

In [5]:
val downloadInfos = runBlocking { track.downloadInfo() }

downloadInfos.forEach { info ->
    println("${info.codec}-${info.bitrateInKbps}")
}

MP3-320
MP3-192


Как можете заметить, мы получаем в различных форматах и битрейтах.

Выбираем определённый формат и скачиваем:

In [6]:
import su.pank.yamapi.downloadInfo.model.Codec

val selected = downloadInfos.first { it.codec == Codec.MP3 && it.bitrateInKbps == 320  }
runBlocking {
    selected.download().take(10)
}

[-1, -5, -32, 68, 0, 3, 54, 51, -124, -68]

**Замечание:** `DownloadInfo` долго не живут (примерно 5 минут), так что получайте `DownloadInfo` перед самим скачиванием трека.

### Получние flac

In [ ]:
runBlocking {
    client.tracks.losslessInfo(track.id)
}